# TP 1 : Using Deep Neural Network to predict risky driver behaviors

In order to improve global road safety and fuel efficiency, predicting driver behaviors using

In this work, we will be using the *Driving Behavior Dataset* by Shashwat Tiwari.

In [1]:
import pandas as pd
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
# Reading the csv file
df = pd.read_csv("features_14.csv")

In [3]:
# Separating the target column and the other columns of the dataset
X = df.drop(columns=["Target"])
Y = df["Target"]

In [4]:
def decrement(x):
  return(x-1)

# Decrementing every value of the class labels so they go from 0 to 3 instead of 1 to 4
Y = Y.apply(decrement)

In [5]:
# Data preprocessing
numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])

preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, df.columns),
            ])

X = preprocessor.fit_transform(df)

In [6]:
# Creating train and test splits for the data and the class labels
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
Y_train, Y_test = train_test_split(Y, test_size=0.2, random_state=42)

In [7]:
# Reshape of the train data
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Definining the input layer
input_layer = Input(shape=(X_train.shape[1:]))

# Number of classes
num_classes = 4

c1 = Conv1D(16, 3, activation='relu', padding='same')(input_layer)
m1 = MaxPooling1D(2, padding='same')(c1)
c2 = Conv1D(8, 3, activation='relu', padding='same')(m1)
m2 = MaxPooling1D(2, padding='same')(c2)
f1 = Flatten()(m2)
d1 = Dense(units=num_classes, activation='softmax')(f1)

# Compiling the model
cnn = Model(input_layer,x)
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
cnn.fit(X_train, Y_train, epochs=10, batch_size=64)

loss, accuracy = cnn.evaluate(X_test, Y_test)




Epoch 1/10


14/14 [==============================] - 1s 3ms/step - loss: 1.4651 - accuracy: 0.2350
Epoch 2/10
14/14 [==============================] - 0s 2ms/step - loss: 1.2112 - accuracy: 0.5199
Epoch 3/10
14/14 [==============================] - 0s 2ms/step - loss: 1.0466 - accuracy: 0.6198
Epoch 4/10
14/14 [==============================] - 0s 2ms/step - loss: 0.8756 - accuracy: 0.8059
Epoch 5/10
14/14 [==============================] - 0s 2ms/step - loss: 0.6943 - accuracy: 0.8865
Epoch 6/10
14/14 [==============================] - 0s 2ms/step - loss: 0.5441 - accuracy: 0.9024
Epoch 7/10
14/14 [==============================] - 0s 2ms/step - loss: 0.4344 - accuracy: 0.9058
Epoch 8/10
14/14 [==============================] - 0s 2ms/step - loss: 0.3545 - accuracy: 0.9115
Epoch 9/10
14/14 [==============================] - 0s 2ms/step - loss: 0.2950 - accuracy: 0.9240
Epoch 10/10
7/7 [==============================] - 0s 2ms/step - loss: 0.2352 - accuracy: 0.9593


In [8]:
print("----Predicted outputs----")
print("Output loss : "+str(loss))
print("Output accuracy : "+ str(accuracy))
print("-------End outputs-------")

----Predicted outputs----
Output loss : 0.23515573143959045
Output accuracy : 0.959276020526886
-------End outputs-------


In [14]:
#On va maintenant créer une matrice de confusion pour une évaluation sur X_test et Y_test
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

predictions = cnn.predict(X_test)
labels = Y_test

cm = confusion_matrix(labels, predictions.argmax(axis=1))
print("---Confusion matrix---")
print(cm)
print("---End confusion matrix---")


7/7 [==============================] - 0s 1ms/step
---Confusion matrix---
[[52  0  1  0]
 [ 0 59  1  0]
 [ 6  0 62  1]
 [ 0  0  0 39]]
---End confusion matrix---


In [15]:
#On récupère l'accuracy, la précision, le recall et le f1-score
from sklearn.metrics import classification_report
print("---Classification report---")
print(classification_report(labels, predictions.argmax(axis=1)))
print("---End classification report---")


---Classification report---
              precision    recall  f1-score   support

           0       0.90      0.98      0.94        53
           1       1.00      0.98      0.99        60
           2       0.97      0.90      0.93        69
           3       0.97      1.00      0.99        39

    accuracy                           0.96       221
   macro avg       0.96      0.97      0.96       221
weighted avg       0.96      0.96      0.96       221

---End classification report---


In [ ]:
#On sauvegarde le modèle, la matrice de confusion et le classification report dans un dossier possédant un timestamp
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs("models/"+timestamp)
cnn.save("models/"+timestamp+"/model")
plt.imshow(cm, cmap='binary')
plt.savefig("models/"+timestamp+"/confusion_matrix.png")
f = open("models/"+timestamp+"/classification_report.txt", "w")
f.write(classification_report(labels, predictions.argmax(axis=1)))
f.close()

print("Model saved in models/"+timestamp+"/model")